### 데이터 받아오기

In [1]:
import json
import numpy as np
import random
import nltk

In [2]:
for i in range(9):
    i = "00"+str(i+1)
    j = "Books_5_"+i+".json"
    with open(j,"r") as data_file:
        raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

 be honest I had a hard time getting into the first story \"The Perfect Secret\". But I am glad I kept going because I wasn't disappointed. I highly recommend this book to anyone who enjoys Amish Fiction.", "overall": 4.0, "summary": "Wonderful Read", "unixReviewTime": 1321315200, "reviewTime": "11 15, 2011"}
 0
{"reviewerID": "AI60470VYBMA1", "asin": "B004LLJ1RW", "reviewerName": "Amazon Customer", "helpful": [0, 0], "reviewText": "I have to admit that I'm surprise this book isn't ranking higher.  I only had one real problem with the book, and I'll write that below.  I liked the book.  I found Dillon to be all that we women dream for in a man with a few flaws that made him irksome, but cute.  I found Cailin to be a bit snobbish at first, but honestly, how many of us would willing let our parents marry us off to a total stranger, them keeping the why's from us, without throwing a fit or two ourselves?  By the end you love her, and she grow 779968


### 데이터 분리하기

In [4]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [5]:
reviewText = []
for i in range(len(review_data)):
    reviewText.append(review_data[i]["reviewText"])

In [6]:
reviewText[9]

'An Amish Wedding by Beth Wiseman, Kathleen Fuller, & Kelly Long was a pure enjoyment to read. Then again I may be biased because each of these ladies are my favorite amish writers. I loved this collaborative piece in which they told the amazing love tales of a group of ladies from one county. Each of the tales was inspiring and touching in the way they came to embrace God and a better understanding of themselves. If I were to go into detail of the relationships, I think I would give away to much of the goodness of the three tales. Read them you will not be disappointed.'

In [7]:
y_data = []
for i in range(len(review_data)):
    y_data.append(review_data[i]["overall"])

In [8]:
from collections import Counter
Counter(y_data)

Counter({1.0: 25979, 2.0: 32363, 3.0: 80554, 4.0: 197497, 5.0: 443574})

<h3>데이터 전처리</h3>

In [9]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [10]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
   
    cleaned_text_list=[]
    for i in range(len(raw_review)):
        
        review_text = BeautifulSoup(raw_review[i]).get_text()
        letters_only = re.sub("[^a-zA-Z]", " ", review_text)
        words = letters_only.lower().split()
        cleaned_text = " ".join(words)
        cleaned_text_list.append(cleaned_text)

    return cleaned_text_list 

In [11]:
pre_data=review_to_words(reviewText)

c:\users\jangy\miniconda3\envs\bi_project\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file c:\users\jangy\miniconda3\envs\bi_project\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [12]:
print(len(reviewText), len(pre_data), len(y_data))

779967 779967 779967


### 데이터 셔플

In [13]:
c = list(zip(pre_data, y_data))
random.shuffle(c)

In [14]:
a, b = zip(*c)

In [15]:
pre_text_data=list(a)
overall_data=np.array(list(b))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True)

tfidf = vectorizer.fit_transform(pre_text_data)

### 가장 작은 평점으로 갯수 맞추기

In [17]:
from collections import Counter
length=sorted(Counter(overall_data).most_common())[0][1]

In [18]:
# pre_data=[]
# overall_data=[]
# for j in range(len(reviewText)):
#     try:
#         if 50<=len(reviewText[j]) or len(reviewText[j])<=1500:
#             pre_data.append(reviewText[j])
#             overall_data.append(y_data[j])
#     except:
#         print(j)

In [19]:
search=[1.0,2.0,3.0,4.0,5.0]
count=0
text=[]
y=[]

for i in search:
    count=0
    for index,value in enumerate(overall_data):
        if (count==length): break
        elif (i==value):
            text.append(pre_text_data[index])
            y.append(value)
            count=count+1

In [20]:
Counter(y)

Counter({1.0: 25979, 2.0: 25979, 3.0: 25979, 4.0: 25979, 5.0: 25979})

In [21]:
print(len(text), len(y))

129895 129895


<h3>데이터 셔플 다시</h3>

In [22]:
c = list(zip(text, y))
random.shuffle(c)

In [23]:
a, b = zip(*c)

In [24]:
text_data=list(a)
overall=np.array(list(b))

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True, stop_words='english', ngram_range=(1,2),
                            max_df = 0.2, min_df =2)


tfidf = vectorizer.fit_transform(text_data)

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf , overall , test_size=0.33, random_state=100)

In [27]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((87029, 953351), (87029,), (42866, 953351), (42866,))

### 예측

### LogisticRegression

In [28]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=0.8, random_state=500, solver='sag', multi_class='multinomial', warm_start=True)

logreg.fit(x_train, y_train)
logreg.predict(x_test)==y_test

# from sklearn import linear_model
# #logreg = linear_model.LogisticRegression()
# logreg = linear_model.LogisticRegression(random_state=42,solver='sag',multi_class='multinomial',warm_start=True)
# logreg.fit(x_train, y_train)

array([ True, False, False, ...,  True,  True, False], dtype=bool)

In [29]:
print(y_test[:10], logreg.predict(x_test)[:10])

[ 4.  5.  3.  1.  4.  3.  1.  5.  3.  2.] [ 4.  4.  2.  2.  4.  4.  1.  5.  1.  1.]


In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)


0.52045910511827553

In [31]:
from sklearn.metrics import precision_score
precision_score(logreg.predict(x_test), y_test,average=None) 

array([ 0.65683487,  0.4353283 ,  0.40615278,  0.45328194,  0.65434833])

In [32]:
from sklearn import metrics
for i in range(1,6):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=i)
#pos_label Label considered as positive and others are considered negative.
    print(i,"점 = ",metrics.auc(fpr, tpr))

1 점 =  0.151521486679
2 점 =  0.30058662371
3 점 =  0.490060079848
4 점 =  0.710184662638
5 점 =  0.844334190332


In [33]:
result=logreg.predict(x_test)

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result))

             precision    recall  f1-score   support

        1.0       0.59      0.66      0.62      8442
        2.0       0.47      0.44      0.45      8605
        3.0       0.47      0.41      0.43      8679
        4.0       0.47      0.45      0.46      8562
        5.0       0.58      0.65      0.61      8578

avg / total       0.51      0.52      0.52     42866



### INPUT 

In [35]:
texts = ["""I enjoyed this book, as long as I kept in mind that the book was written for young adults.  
         The characters were believable and it was easy to identify with their pain.""",
        "I think the author gets lost in his own thoughts and trys to make storys of something that's not there.",
        """I was very excited to start reading this book and although I did like it, 
        I thought that a lot of the words to describe body parts was kind of high-schoolish.  
        Didn't care for the cliff hanging ending.""",
        """I don't know if you are familiar with FANFIC on the internet but that what this book reminded me of.  
        I love the series and thought I would like the books...not so much.""",
        """As I love western era romances, this one was wonderful.  An easy read and ended the way we always hope....happily!"""]
overall=[4,1,3,2,5]
vecs = vectorizer.transform(review_to_words(texts)) 

c:\users\jangy\miniconda3\envs\bi_project\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file c:\users\jangy\miniconda3\envs\bi_project\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [36]:
print(overall,logreg.predict(vecs))

[4, 1, 3, 2, 5] [ 4.  2.  2.  5.  5.]
